<a href="https://colab.research.google.com/github/edramos-lab/Vision-Transfomers/blob/main/Baseplates_DeiT_k_folded_deit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install wandb --upgrade
!pip install tensorflow-addons==0.8.3
!pip install tensorflow
!pip install roboflow
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-addons==0.8.3 (from versions: 0.16.1, 0.17.0, 0.17.1, 0.18.0, 0.19.0, 0.20.0)
ERROR: No matching distribution found for tensorflow-addons==0.8.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import KFold


import torchvision.datasets as datasets
from torch.utils.data import DataLoader,random_split,SubsetRandomSampler
import wandb
import numpy as np
import timm
# Import W&B
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import os
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset, DataLoader, random_split, SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import KFold
os.environ["WANDB_START_METHOD"] = "thread"

In [54]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Zj3z1BE0KBeUDmEdARjg")
project = rf.workspace("edgar-rene-ramos-acosta-uabc-edu-mx").project("nigel-chassises")
dataset = project.version(1).download("folder")



#from roboflow import Roboflow
#rf = Roboflow(api_key="wv1RWb130ECTfhxSNxPS")
#project = rf.workspace("footdiseaseimgclass").project("things-jam67")
#dataset = project.version(1).download("folder")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to nigel-chassises-1 in folder:: 100%|██████████| 6103/6103 [00:01<00:00, 4476.52it/s]


In [55]:
import os

path = '/content/nigel-chassises-1/train'
subfolders = [f.name for f in os.scandir(path) if f.is_dir()]
class_names = sorted(subfolders)

print(class_names)


['Class1', 'Class2', 'Class3', 'Class4', 'Class5', 'Class6']


In [56]:
# Define the data transforms
# Define the transform for the input images
IMG_SIZE=224

train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),  # resize the input image
    transforms.ToTensor(),  # convert to tensor
    transforms.ColorJitter(0.5,0.5,0.5),
    transforms.RandomAffine(degrees=20,translate=None, scale=None, shear=20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # normalize the input image
])

valid_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

    
test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]) 
"""
# Define the transform for the input images
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # resize the input image
    transforms.ToTensor(),  # convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # normalize the input image
])

valid_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

    
test_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
"""

train_data= datasets.ImageFolder('/content/nigel-chassises-1/train', transform=train_transforms)
valid_data = datasets.ImageFolder('/content/nigel-chassises-1/valid', transform=valid_transforms)
test_data = datasets.ImageFolder('/content/nigel-chassises-1/test', transform=test_transforms)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)


In [57]:
# Define the train-validation split ratio
train_ratio = .75
val_ratio = 1-train_ratio

# Define the indices of the full dataset
dataset_size = len(train_data)
indices = list(range(dataset_size))

# Split the indices into train and validation indices
train_size = int(train_ratio * dataset_size)
val_size = dataset_size - train_size
train_indices, val_indices = random_split(indices, [train_size, val_size], generator=torch.Generator().manual_seed(42))

# Create a Subset of the ImageFolder dataset using the train_indices
train_subset = Subset(train_data, train_indices)

# Split the train subset into train and validation sets
train_subset_size = len(train_subset)
train_indices_subset = list(range(train_subset_size))
train_size_subset = int(train_ratio * train_subset_size)
val_size_subset = train_subset_size - train_size_subset
train_indices_subset, val_indices_subset = random_split(train_indices_subset, [train_size_subset, val_size_subset], generator=torch.Generator().manual_seed(42))

# Create SubsetRandomSampler for train and validation indices
train_sampler = SubsetRandomSampler(train_indices_subset)
val_sampler = SubsetRandomSampler(val_indices_subset)

In [58]:
train_data=train_subset

In [59]:
def testing(test_loader):  
  y_pred=[]
  y_scores=[]
  y_true=[]
  all_preds=[]
  all_targets=[]
  with torch.no_grad():
      for images, labels in test_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, preds = torch.max(outputs.data, 1)

                  # append the labels to the lists
          y_true += labels.tolist()
          y_pred += preds.tolist()
                  # Append the preds scores and true labels to the lists
          all_preds.extend(preds.cpu().numpy())
                    # append the predicted scores to the list
          y_scores += torch.softmax(outputs, dim=1).tolist()
          all_targets.extend(labels.cpu().numpy())


  # Convert the lists to NumPy arrays
  all_preds = np.array(all_preds)
  all_targets = np.array(all_targets)

  from sklearn.metrics import classification_report,precision_score,accuracy_score,recall_score,f1_score,cohen_kappa_score,matthews_corrcoef, confusion_matrix,roc_curve,auc
  import seaborn as sns
  import matplotlib.pyplot as plt

  cm = confusion_matrix(all_targets, all_preds)
  
  #sns.heatmap(cm, annot=True, cmap='Blues')
  # plot the confusion matrix as a heatmap
    # Create a Matplotlib figure with class names
  fig, ax = plt.subplots()
  im = ax.imshow(cm, interpolation='nearest', cmap='Blues')
  ax.set(xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        xticklabels=class_names, yticklabels=class_names,
        title='Confusion matrix',
        ylabel='True label',
        xlabel='Predicted label')
  plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
          rotation_mode="anchor")
  fmt = 'd'
  thresh = cm.max() / 2.
  for i in range(cm.shape[0]):
      for j in range(cm.shape[1]):
          ax.text(j, i, format(cm[i, j], fmt),
                  ha="center", va="center",
                  color="white" if cm[i, j] > thresh else "black")
  fig.tight_layout()


  # log the confusion matrix to WandB
  wandb.log({'confusion_matrix': wandb.Image(fig)})
  # ROC
  wandb.log({"roc": wandb.plots.ROC(y_true, y_scores, labels=class_names)})

  # Precision Recall
  wandb.log({'pr': wandb.plots.precision_recall(y_true, y_scores, class_names)})

  # Learning Curve
  #wandb.sklearn.plot_learning_curve(model, images, labels)

  # Confusion Matrix
  wandb.log({'CM': wandb.sklearn.plot_confusion_matrix(y_true, y_pred, labels=class_names)})


  test_accuracy_score=accuracy_score(y_true,y_pred)
  test_precision_score=precision_score(y_true, y_pred,average='macro')
  test_recall_score=recall_score(y_true,y_pred,average='macro')
  test_f1_score=f1_score(y_true,y_true,average='macro')
  test_cohen_kapp=cohen_kappa_score(y_true,y_pred)
  test_matthews_corrcoef=matthews_corrcoef(y_true,y_pred)

  report = classification_report(y_true, y_pred, target_names=class_names)
  print(report)

  # compute TP, TN, FP, and FN counts for each class
  TP = cm.diagonal()
  TN = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - TP)
  FP = np.sum(cm, axis=0) - TP
  FN = np.sum(cm, axis=1) - TP

  # print the results
  for i in range(num_classes):
      print(f"Class {i}: TP={TP[i]}, TN={TN[i]}, FP={FP[i]}, FN={FN[i]}")
  # Log metrics to wandb
  wandb.log({'True Positives:': sum(TP), 'True Negative: ':sum(TN), 'False positive:': sum(FP), 'False Negative': sum(FN)})
  wandb.log({'test_accuracy_score': test_accuracy_score,'test_precision_score': test_precision_score, 'test_recall_score':test_recall_score, 'test_f1_score': test_f1_score, 'test_cohen_kapp': test_cohen_kapp, 'test_matthews_corrcoef':test_matthews_corrcoef})


In [60]:
MODEL_NAME=['deit_base_patch16_224', 'deit_tiny_patch16_224', 'deit_small_patch16_224', 'deit_tiny_distilled_patch16_224', 'deit_small_distilled_patch16_224', 'deit_base_distilled_patch16_224']


In [61]:
MODEL=MODEL_NAME[3]

In [62]:

# Set up the model
# Define the model
#model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)

model = torch.hub.load('facebookresearch/deit:main', MODEL, pretrained=True)

# Replace the last layer of the model to output 6 classes instead of 1000




epochs = 3
batch_size = 64

# Replace the last layer of the model to output 11 classes instead of 1000
num_classes = len(class_names)

model.head = nn.Linear(in_features=model.head.in_features, out_features=num_classes, bias=True)
model.head_dist=nn.Linear(in_features=model.head.in_features, out_features=num_classes, bias=True)
# Learning rate
LEARNING_RATE=2e-4
# Define the device to use
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)



Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main


In [ ]:
# Set up k-fold cross-validation
k = len(class_names)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Train the model with k-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(train_data)):
  print(f'Fold [{fold+1}/{k}]')
  project = "Zinc-Plated-Components-Classifier"
  config = {"batch_size": batch_size, "epochs": epochs, "architecture": MODEL,'LEARNING_RATE': LEARNING_RATE}
  wandb.init(entity="aimfg-california",project=project,mode="online",config=config)

  train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
  
  val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

  train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
  print(len(train_loader))
  val_loader = DataLoader(train_data, batch_size=64, sampler=val_sampler)

  for epoch in range(epochs):
      #model.train()
      for images, labels in train_loader:
          optimizer.zero_grad()
          images = images.to(device)
          labels = labels.to(device)
          if MODEL=='deit_tiny_distilled_patch16_22' or MODEL=='deit_small_distilled_patch16_224':
            outputs, _ = model(images)  # assuming that the first element of the tuple is the relevant tensor
          else:  
            outputs = model(images) # use the first tensor in the tupleoutputs = model(images)
            if isinstance(outputs, tuple):
              output = torch.cat(outputs, dim=1)
            else:
              output = outputs
          #output = torch.cat(outputs, dim=1)
          loss = criterion(output, labels)
          loss.backward()
          optimizer.step()

      model.eval()
      with torch.no_grad():
          total_loss, total_correct = 0, 0
          for images, labels in val_loader:
              images = images.to(device)
              labels = labels.to(device)
              if MODEL=='deit_tiny_distilled_patch16_22' or MODEL=='deit_small_distilled_patch16_224':
                outputs, _ = model(images)  # assuming that the first element of the tuple is the relevant tensor
              else:  
                outputs = model(images) # use the first tensor in the tupleoutputs = model(images)
              #output = torch.cat(outputs, dim=1)
              loss = criterion(outputs, labels)
              total_loss += loss.item() * images.size(0)
              total_correct += (outputs.argmax(1) == labels).sum().item()

          val_loss = total_loss / len(val_loader.sampler)
          val_acc = total_correct / len(val_loader.sampler)

      print(f'Epoch [{epoch+1}/{epochs}], Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
            # Log metrics to wandb

      wandb.log({'K-Fold': fold+1, 'Train_datasize': len(train_idx), 'Valid_datasize': len(val_idx), 'Val_loss': val_loss,'Val_Accuracy': val_acc})
            

  # Save the model weights for this fold
  #torch.save(model.state_dict(), f'deit_fold_{fold}.pth')
  testing(test_loader)
   



Fold [1/6]


False Negative,▁
False positive:,▁
K-Fold,▁▁▁
Train_datasize,▁▁▁
True Negative:,▁
True Positives:,▁
Val_Accuracy,▂▁█
Val_loss,▇█▁
Valid_datasize,▁▁▁
test_accuracy_score,▁
test_cohen_kapp,▁


48
Epoch [1/3], Val Loss: 0.5551, Val Acc: 0.7977
Epoch [2/3], Val Loss: 0.1348, Val Acc: 0.9589
Epoch [3/3], Val Loss: 0.2616, Val Acc: 0.9095
              precision    recall  f1-score   support

      Class1       0.90      1.00      0.95        98
      Class2       1.00      0.76      0.86        90
      Class3       0.83      0.94      0.88       101
      Class4       0.88      0.84      0.86       113
      Class5       1.00      0.98      0.99       113
      Class6       0.96      1.00      0.98        93

    accuracy                           0.92       608
   macro avg       0.93      0.92      0.92       608
weighted avg       0.93      0.92      0.92       608

Class 0: TP=98, TN=499, FP=11, FN=0
Class 1: TP=68, TN=518, FP=0, FN=22
Class 2: TP=95, TN=487, FP=20, FN=6
Class 3: TP=95, TN=482, FP=13, FN=18
Class 4: TP=111, TN=495, FP=0, FN=2
Class 5: TP=93, TN=511, FP=4, FN=0
Fold [2/6]


False Negative,▁
False positive:,▁
K-Fold,▁▁▁
Train_datasize,▁▁▁
True Negative:,▁
True Positives:,▁
Val_Accuracy,▁█▆
Val_loss,█▁▃
Valid_datasize,▁▁▁
test_accuracy_score,▁
test_cohen_kapp,▁


48
Epoch [1/3], Val Loss: 0.0991, Val Acc: 0.9720
Epoch [2/3], Val Loss: 0.0632, Val Acc: 0.9868
Epoch [3/3], Val Loss: 0.0810, Val Acc: 0.9819
              precision    recall  f1-score   support

      Class1       1.00      0.97      0.98        98
      Class2       0.99      1.00      0.99        90
      Class3       0.98      0.99      0.99       101
      Class4       1.00      0.99      1.00       113
      Class5       0.98      1.00      0.99       113
      Class6       1.00      1.00      1.00        93

    accuracy                           0.99       608
   macro avg       0.99      0.99      0.99       608
weighted avg       0.99      0.99      0.99       608

Class 0: TP=95, TN=510, FP=0, FN=3
Class 1: TP=90, TN=517, FP=1, FN=0
Class 2: TP=100, TN=505, FP=2, FN=1
Class 3: TP=112, TN=495, FP=0, FN=1
Class 4: TP=113, TN=493, FP=2, FN=0
Class 5: TP=93, TN=515, FP=0, FN=0
Fold [3/6]


False Negative,▁
False positive:,▁
K-Fold,▁▁▁
Train_datasize,▁▁▁
True Negative:,▁
True Positives:,▁
Val_Accuracy,▁█▆
Val_loss,█▁▄
Valid_datasize,▁▁▁
test_accuracy_score,▁
test_cohen_kapp,▁


48


In [ ]:
wandb.finish()